# Tutorial

## Prerequisites.

#### Download and install Miniconda, Anaconda, or Conda.

https://docs.conda.io/en/latest/miniconda.html

### Create the Conda environment.

```
conda env create --file conda\win.yml
conda activate tyche
pip install mip
```

#### Activate the Tyche environment.

```
conda activate tyche
```

## Set up.

### Import packages.

#### Import the system packages.

In [1]:
import os
import sys

#### Add the main Tyche packages to the search path.

In [2]:
sys.path.insert(1, os.path.abspath("../src"))

#### Import tyche and related packages.

In [3]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

## Example problem: wind turbines.

Here is a highly simplified and approximate model for a wind turbine.

### Environmental constants.

*   Wind sheer exponent: $\alpha = 0.16$

### Scale of operations.

*   Project scale: $S$.
*   Rotor diameter: $R = 70 \cdot S^\frac{1}{2+3\alpha} ~\textrm{m}$.
*   Hub height: $H = 65 \cdot S^\frac{1}{2+3\alpha} ~\textrm{m}$.
*   Machine rating: $M = 1500 \cdot S ~\textrm{kW}$. 

### Production.

*   Annual energy production: $4312 \cdot S ~\textrm{MWh}$

### Design parameters.

*   Rotor design: $\rho \in [0, 1]$, with $\rho_0 = 0$.
*   Drive design: $\delta \in [0.8, 0.90]$, with $\delta_0 = 0.90$.
*   Tower design: $\tau \in [0.5, 1.0]$, with $\tau_0 = 1.0$.
*   BOS factor: $\beta \in [0.5, 1.0]$, with $\beta_0 = 1.0$.
*   O&M factor: $\mu \in [0.5, 1.0]$, with $\mu_0 = 1.0$.

### Costs.

*   Rotor: $1.6 \cdot R^{2.8} - 60000 \cdot \rho ~\textrm{USD}$.
*   Drive, nacelle, systems: $900 \cdot M^\delta ~\textrm{USD}$.
*   Tower: $0.015 \cdot R^{2.8} \cdot H^\tau ~\textrm{USD}$.
*   Balance of system: $250 \cdot \beta \cdot M ~\textrm{USD}$.
*   Replacements: $10.6 \cdot M ~\textrm{USD}$.
*   O&M: $20 \cdot \mu \cdot M ~\textrm{USD}$.
*   Land lease: $3.5 \cdot M ~\textrm{USD}$.

### Metrics.

*   Capacity factor: $32.82~\%$

## Design the technology model.

### Technology name.

Choose a unique name for the technology being modeled. *In this example we'll call the technology `Wind Turbine`.*

Also choose a unique name for the default, reference, or base-case scenario. *In this example, we'll call the default scenario `Reference`.*

### Dimensions and indices.

Decide which quantities will be tracked as indices in the model, and settle on their units of measurment and default values:
1.  Types of capital costs. *In this example we have . . .*
    *   Rotor
    *   Drive
    *   Tower
    *   BOS
2.  Types of fixed cost. *In this example we have . . .*
    *   Replacements
    *   O&M
    *   Land
3.  Inputs to the process. *In this example we have . . .*
    *   Wind
4.  Outputs from the process. *In this example we have . . .*
    *   Electricity
5.  Metrics. *In this example we have . . .*
    *   Cost
    *   LCOE
    *   CF
    *   AEP
6.  Parameters. *In this example we have . . .*
    *   Rotor design ($\rho$)
    *   Drive design ($\delta$)
    *   Tower design ($\tau$)
    *   BOS factor ($\beta$)
    *   O&M factor ($\mu$)

Note that in each category, the numeric indices for each item are numbered with integers starting from zero.

See the [model](https://tyche.live/doc-src/formulation.html) and [database](https://tyche.live/doc-src/database.html) documentation for more details.

### Create the `indices` table.

Enter the data for your model in the tab-delimited-value file [tutorial/data/indices.tsv](data/).

Check to see that the data file reads correctly:

In [4]:
my_designs = ty.Designs("data")
my_designs.indices.reset_index("Index").sort_values(["Type", "Offset"])

,,Index,Offset,Description,Notes
Technology,Type,,,,


Check your `indices` table against the following:

In [5]:
answers_designs = ty.Designs("answers")
answers_designs.indices.reset_index("Index").sort_values(["Type", "Offset"])

Index  Offset  \
Technology   Type                            
Wind Turbine Capital         Rotor       0   
             Capital         Drive       1   
             Capital         Tower       2   
             Capital           BOS       3   
             Fixed    Replacements       0   
             Fixed             O&M       1   
             Fixed            Land       2   
             Input            Wind       0   
             Metric           Cost       0   
             Metric           LCOE       1   
             Metric             CF       2   
             Metric            AEP       3   
             Output    Electricity       0   

                                                            Description Notes  
Technology   Type                                                              
Wind Turbine Capital        Rotor blades, hub, bearings, spinner, nose.        
             Capital  Drive, gearbox, bearings, electronics, frame, ...        
             Capital                                         The tower.        
             Capital                    Foundations, roads, civil work.        
             Fixed                              Replacement components.        
             Fixed                          Operations and maintenance.        
             Fixed                                          Land lease.        
             Input    Just a placeholder because at least one input ...        
             Metric                           Cost of the wind turbine.        
             Metric                           Levelized cost of energy.        
             Metric                                    Capacity factor.        
             Metric                           Annual energy production.        
             Output                               Annual energy output.

### Create the `results` table.

Enter the data for the units of measure in your model in the tab-delimited-value file [tutorial/data/results.tsv](data/).

Check to see that the data file reads correctly:

In [6]:
my_designs = ty.Designs("data")
my_designs.results

,,,Units,Notes
Technology,Variable,Index,,


Check your `results` table against the following:

In [7]:
answers_designs = ty.Designs("answers")
answers_designs.results

Units Notes
Technology   Variable Index                              
Wind Turbine Cost     Cost                $/turbine      
             Metric   AEP          MWh/turbine/year      
                      CF                      %/100      
                      Cost           $/turbine/year      
                      LCOE                    $/kWh      
             Output   Electricity       kWh/turbine

### Create the `designs` table.

Enter the data for your model in the tab-delimited-value file [tutorial/data/designs.tsv](data/).

Check to see that the data file reads correctly:

In [8]:
my_designs = ty.Designs("data")
my_designs.designs.reset_index(["Variable", "Index"]).sort_values(["Variable", "Index"])

,,Variable,Index,Value,Units,Notes
Technology,Scenario,,,,,


Check your `designs` table against the following:

In [9]:
answers_designs = ty.Designs("answers")
answers_designs.designs.xs("Reference", level = "Scenario", drop_level = False).reset_index(["Variable", "Index"]).sort_values(["Variable", "Index"])

Variable        Index   Value  Units  \
Technology   Scenario                                                   
Wind Turbine Reference              Input         Wind    10.0    m/s   
             Reference   Input efficiency         Wind   0.593  %/100   
             Reference        Input price         Wind       0    n/a   
             Reference           Lifetime          BOS     8.6   year   
             Reference           Lifetime        Drive     8.6   year   
             Reference           Lifetime        Rotor     8.6   year   
             Reference           Lifetime        Tower     8.6   year   
             Reference  Output efficiency  Electricity  0.3282  %/100   
             Reference       Output price  Electricity       0  $/kWh   
             Reference              Scale          NaN       1  %/100   

                                                                    Notes  
Technology   Scenario                                                      
Wind Turbine Reference                           Wind speed at 50 meters.  
             Reference                                          Betz law.  
             Reference                                      Wind is free.  
             Reference         Reciprocal of the capital-recovery factor.  
             Reference         Reciprocal of the capital-recovery factor.  
             Reference         Reciprocal of the capital-recovery factor.  
             Reference         Reciprocal of the capital-recovery factor.  
             Reference                                   Capacity factor.  
             Reference                              Not tracking revenue.  
             Reference  Scale of operations, relative to a 1.5MW turbine.

### Create the `parameters` table.

Enter the data for your model in the tab-delimited-value file [tutorial/data/parameters.tsv](data/).

Check to see that the data file reads correctly:

In [10]:
my_designs = ty.Designs("data")
my_designs.parameters.reset_index("Parameter").sort_values("Offset")

,,Parameter,Offset,Value,Units,Notes
Technology,Scenario,,,,,


Check your `parameters` table against the following:

In [11]:
answers_designs = ty.Designs("answers")
answers_designs.parameters.xs("Reference", level = "Scenario", drop_level = False).reset_index("Parameter").sort_values("Offset")

Parameter  Offset Value Units  Notes
Technology   Scenario                                          
Wind Turbine Reference  Rotor design       0     0     1    rho
             Reference  Drive design       1  0.90     1  delta
             Reference  Tower design       2     1     1    tau
             Reference    BOS factor       3     1     1   beta
             Reference    O&M factor       4     1     1     mu

## Implement the technology model.

The implementation of a technology model consists of a capital cost function, a fixed cost function, a production function, and a metrics function.

See the [model](https://tyche.live/doc-src/formulation.html) documentation for more details.

The [src/technology/](../src/technology/) folder has examples of several technology models.

#### Capital cost function.

The capital cost function takes the scale of the operations and the array of technology parameters as arguments and it returns the capital costs for the technology.

Implement the capital cost function for your technology in the file [tutorial/my_technology.py](./).

In [12]:
def capital_cost(scale, parameter):
  """
  Capital cost function.

  Parameters
  ----------
  scale : float
    The scale of operation.
  parameter : array
    The technological parameterization.
  """

  # Stack the costs for each category into a single array that we return.
  return np.stack([
      
  ])

Compare your function to the following:

In [13]:
def answers_capital_cost(scale, parameter):
  """
  Capital cost function.

  Parameters
  ----------
  scale : float
    The scale of operation.
  parameter : array
    The technological parameterization.
  """

  # We aren't varying the wind sheer exponent.
  alpha = 0.16

  # It is handy to copy the elements of the parameter array into meaningful variable names.
  rho   = parameter[0]
  delta = parameter[1]
  tau   = parameter[2]
  beta  = parameter[3]
  mu    = parameter[4]

  # Compute the rotor diamter, hub height, and machine rating.
  r = 70 * scale**(1 / (2 + 3 * alpha))
  h = 65 * scale**(1 / (2 + 3 * alpha))
  m = 1500 * scale

  # Compute the components of capital cost.
  rotor = 1.6 * r**2.8 - 60000 * rho
  drive = 900 * m**delta
  tower = 0.015 * r**2.8 * h**tau
  bos   = 250 * beta * m

  # Stack the costs for each category into a single array that we return.
  return np.stack([
      rotor,
      drive,
      tower,
      bos,
  ])

Test this for the reference case:

In [14]:
example_capital = answers_capital_cost(1, [0, 0.9, 1, 1, 1])
example_capital

array([234635.91808147, 649717.61674841, 142981.2625809 , 375000.        ])

Interpretation:
*   Rotor: \$235k
*   Drive: \$649k
*   Tower: \$143k
*   BOS: \$375k

#### Fixed cost function.

The fixed cost function takes the scale of the operations and the array of technology parameters as arguments and it returns the fixed costs for the technology.

Implement the fixed cost function for your technology in the file [tutorial/my_technology.py](./).

In [15]:
def fixed_cost(scale, parameter):
  """
  Capital cost function.

  Parameters
  ----------
  scale : float
    The scale of operation.
  parameter : array
    The technological parameterization.
  """
  # Stack the costs for each category into a single array that we return.
  return np.stack([

  ])

Compare your function to the following:

In [16]:
def answers_fixed_cost(scale, parameter):
  """
  Capital cost function.

  Parameters
  ----------
  scale : float
    The scale of operation.
  parameter : array
    The technological parameterization.
  """

  # We aren't varying the wind sheer exponent.
  alpha = 0.16

  # It is handy to copy the elements of the parameter array into meaningful variable names.
  rho   = parameter[0]
  delta = parameter[1]
  tau   = parameter[2]
  beta  = parameter[3]
  mu    = parameter[4]

  # Compute the rotor diamter, hub height, and machine rating.
  r = 70 * scale**(1 / (2 + 3 * alpha))
  h = 65 * scale**(1 / (2 + 3 * alpha))
  m = 1500 * scale

  # Compute the components of fixed cost.
  replacement            = 10.6 * m
  operations_maintenance = 20 * mu * m
  land_lease             = 3.5 * m

  # Stack the costs for each category into a single array that we return.
  return np.stack([
      replacement,
      operations_maintenance,
      land_lease
  ])

Test this for the reference case:

In [17]:
example_fixed = answers_fixed_cost(1, [0, 0.9, 1, 1, 1])
example_fixed

array([15900., 30000.,  5250.])

Interpretation:
*   Replacement: \$16k/yr
*   O&M: \$30k/yr
*   Land: \$5k/yr

#### Production function.

The production function takes the scale of the operations, the capital costs, the lifetime, the fixed costs, and the array of technology parameters as arguments and it returns the production (outputs) for the technology.

Implement the production function for your technology in the file [tutorial/my_technology.py](./).

In [18]:
def production(scale, capital, lifetime, fixed, input, parameter):
  """
  Production function.

  Parameters
  ----------
  scale : float
    The scale of operation.
  capital : array
    Capital costs.
  lifetime : float
    Technology lifetime.
  fixed : array
    Fixed costs.
  input : array
    Input quantities. 
  parameter : array
    The technological parameterization.
  """

  # Stack the output for each category into a single array that we return.
  return np.stack([

  ])

Compare your function to the following:

In [19]:
def answers_production(scale, capital, lifetime, fixed, input, parameter):
  """
  Production function.

  Parameters
  ----------
  scale : float
    The scale of operation.
  capital : array
    Capital costs.
  lifetime : float
    Technology lifetime.
  fixed : array
    Fixed costs.
  input : array
    Input quantities. 
  parameter : array
    The technological parameterization.
  """

  # We aren't varying the wind sheer exponent.
  alpha = 0.16

  # It is handy to copy the elements of the parameter array into meaningful variable names.
  rho   = parameter[0]
  delta = parameter[1]
  tau   = parameter[2]
  beta  = parameter[3]
  mu    = parameter[4]

  # Compute the production of electricity.
  electricity = 4312 * scale / 0.3282

  # Stack the output for each category into a single array that we return.
  return np.stack([
      electricity,
  ])

Test this for the reference case:

In [20]:
example_production = answers_production(
    1,
    example_capital,
    [8.6, 8.6, 8.6],
    example_fixed,
    [0],
    [0, 0.9, 1, 1, 1],
)
example_production

array([13138.33028641])

Interpretation:
*   Rated annual production: 13 GW

#### Metric function.

The metric function takes information on costs, inputs, outputs, and parameters and it returns the metrics for the technology.

Implement the metrics function for your technology in the file [tutorial/my_technology.py](./).

In [21]:
def metrics(scale, capital, lifetime, fixed, input_raw, input, output_raw, output, cost, parameter):
  """
  Metrics function.

  Parameters
  ----------
  scale : float
    The scale of operation.
  capital : array
    Capital costs.
  lifetime : float
    Technology lifetime.
  fixed : array
    Fixed costs.
  input_raw : array
    Raw input quantities (before losses).
  input : array
    Input quantities. 
  output_raw : array
    Raw output quantities (before losses).
  output : array
    Output quantities. 
  cost : array
    Costs.
  parameter : array
    The technological parameterization.
  """

  # Package results.
  return np.stack([

  ])

Compare your function to the following:

In [22]:
def answers_metrics(scale, capital, lifetime, fixed, input_raw, input, output_raw, output, cost, parameter):
  """
  Metrics function.

  Parameters
  ----------
  scale : float
    The scale of operation.
  capital : array
    Capital costs.
  lifetime : float
    Technology lifetime.
  fixed : array
    Fixed costs.
  input_raw : array
    Raw input quantities (before losses).
  input : array
    Input quantities. 
  output_raw : array
    Raw output quantities (before losses).
  output : array
    Output quantities. 
  cost : array
    Costs.
  parameter : array
    The technological parameterization.
  """

  # We aren't varying the wind sheer exponent.
  alpha = 0.16

  # It is handy to copy the elements of the parameter array into meaningful variable names.
  rho   = parameter[0]
  delta = parameter[1]
  tau   = parameter[2]
  beta  = parameter[3]
  mu    = parameter[4]

  # Compute the metrics.
  lcoe = cost / output[0] / 1000
  cf = output[0] / output_raw[0]
  aep = output[0]

  # Package results.
  return np.stack([
    cost,
    lcoe,
    cf  ,
    aep ,
  ])

Test this for the reference case:

In [23]:
example_metrics = answers_metrics(
    1,
    example_capital,
    [8.6, 8.6, 8.6],
    example_fixed,
    [0],
    [0],
    example_production,
    example_production * 0.3282,
    sum(example_capital / 8.6) + sum(example_fixed),
    [0, 0.9, 1, 1, 1],
)
example_metrics

array([2.14212186e+05, 4.96781507e-02, 3.28200000e-01, 4.31200000e+03])

Interpretation:
*   Levelized cost: \$214k/yr
*   LCOE: \$0.05/kWh
*   Capacity factor: 32.82%
*   Annual energy production: 4312 MWh/yr

### Create the `functions` table.

Enter the technology name and any notes for your model in the tab-delimited-value file [tutorial/data/functions.tsv](data/). You can also edit the module or function names in this table, if you changed them.

Check to see that the data file reads correctly:

In [24]:
my_designs = ty.Designs("data")
my_designs.functions

,Style,Module,Capital,Fixed,Production,Metrics,Notes
Technology,,,,,,,
My Technology,numpy,my_technology,capital_cost,fixed_cost,production,metrics,my notes


Compare your `functions` table to the following:

In [25]:
answers_designs = ty.Designs("answers")
answers_designs.functions

,Style,Module,Capital,Fixed,Production,Metrics,Notes
Technology,,,,,,,
Wind Turbine,numpy,answers_technology,capital_cost,fixed_cost,production,metrics,


## Create investment for reference case.

### Create the `tranches` table.

To get started, we just want to create on tranche and one category for the base case or reference scenario.

Edit the name of the reference case for your model in the tab-delimited-value file [tutorial/data/tranches.tsv](data/).

Check to see that the data file reads correctly:

In [26]:
my_investments = ty.Investments("data")
my_investments.tranches

,,,Amount,Notes
Category,Tranche,Scenario,,
Example Category,Example Tranche,Base Case,0.0,


Compare your `tranches` table to the following:

In [27]:
answers_investments = ty.Investments("answers")
answers_investments.tranches

,,,Amount,Notes
Category,Tranche,Scenario,,
No Category,Business as Usual,Reference,0.0,base case


### Create the `investments` table.

To get started, we just want to create one investment for the base case or reference scenario.

Edit the name of the reference case for your model in the tab-delimited-value file [tutorial/data/investments.tsv](data/).

Check to see that the data file reads correctly:

In [28]:
my_investments = ty.Investments("data")
my_investments.investments

,,,Notes
Investment,Category,Tranche,
Example Investment,Example Category,Example Tranche,


Compare your `investments` table to the following:

In [29]:
answers_investments = ty.Investments("answers")
answers_investments.investments

,,,Notes
Investment,Category,Tranche,
No Investment,No Category,Business as Usual,


## Simulate the base case or reference scenario.

### Load the data.

In [30]:
answers_designs = ty.Designs("answers")

In [31]:
answers_investments = ty.Investments("answers")

### Compile the production and metric functions for each technology in the dataset.

In [32]:
answers_designs.compile()

### Compute costs and metrics for tranches.

Tranches are atomic units for building investment portfolios. Evaluate all of the tranches, so we can assemble them into investments (portfolios).

In [33]:
tranche_results = answers_investments.evaluate_tranches(answers_designs, sample_count=5000)

The tranche amounts are simple how much each tranche costs.

In [34]:
tranche_results.amounts

,,Amount
Category,Tranche,
No Category,Business as Usual,0.0


The tranch metrics show the values of each metric for each member of the ensemble of simulations.

In [35]:
tranche_results.metrics

Value  \
Category    Tranche           Scenario  Sample Technology   Index                  
No Category Business as Usual Reference 1      Wind Turbine AEP      4312.000000   
                                                            CF          0.328200   
                                                            Cost   214212.185745   
                                                            LCOE        0.049678   
                                        2      Wind Turbine AEP      4312.000000   
...                                                                          ...   
                                        4999   Wind Turbine LCOE        0.049678   
                                        5000   Wind Turbine AEP      4312.000000   
                                                            CF          0.328200   
                                                            Cost   214212.185745   
                                                            LCOE        0.049678   

                                                                              Units  
Category    Tranche           Scenario  Sample Technology   Index                    
No Category Business as Usual Reference 1      Wind Turbine AEP    MWh/turbine/year  
                                                            CF                %/100  
                                                            Cost     $/turbine/year  
                                                            LCOE              $/kWh  
                                        2      Wind Turbine AEP    MWh/turbine/year  
...                                                                             ...  
                                        4999   Wind Turbine LCOE              $/kWh  
                                        5000   Wind Turbine AEP    MWh/turbine/year  
                                                            CF                %/100  
                                                            Cost     $/turbine/year  
                                                            LCOE              $/kWh  

[20000 rows x 2 columns]

In [36]:
tranche_results.metrics.groupby("Index").aggregate(np.median)

,Value
Index,
AEP,4312.000000
CF,0.328200
Cost,214212.185745
LCOE,0.049678


### Compute costs and metrics for investments.

Now evaluate the investments.

In [37]:
investment_results = answers_investments.evaluate_investments(answers_designs, sample_count=50)

The investment amounts are simple how much each tranche costs.

In [38]:
investment_results.amounts

,Amount
Investment,
No Investment,0.0


The investment metrics show the values of each metric for each member of the ensemble of simulations.

In [39]:
investment_results.metrics

Value  \
Investment    Category    Tranche           Scenario  Sample Technology   Index                  
No Investment No Category Business as Usual Reference 1      Wind Turbine AEP      4312.000000   
                                                                          CF          0.328200   
                                                                          Cost   214212.185745   
                                                                          LCOE        0.049678   
                                                      2      Wind Turbine AEP      4312.000000   
...                                                                                        ...   
                                                      49     Wind Turbine LCOE        0.049678   
                                                      50     Wind Turbine AEP      4312.000000   
                                                                          CF          0.328200   
                                                                          Cost   214212.185745   
                                                                          LCOE        0.049678   

                                                                                            Units  
Investment    Category    Tranche           Scenario  Sample Technology   Index                    
No Investment No Category Business as Usual Reference 1      Wind Turbine AEP    MWh/turbine/year  
                                                                          CF                %/100  
                                                                          Cost     $/turbine/year  
                                                                          LCOE              $/kWh  
                                                      2      Wind Turbine AEP    MWh/turbine/year  
...                                                                                           ...  
                                                      49     Wind Turbine LCOE              $/kWh  
                                                      50     Wind Turbine AEP    MWh/turbine/year  
                                                                          CF                %/100  
                                                                          Cost     $/turbine/year  
                                                                          LCOE              $/kWh  

[200 rows x 2 columns]

A summary of the metrics is available at the investment level rather than at the tranche level.

In [40]:
investment_results.summary

Value             Units
Investment    Sample Index                                 
No Investment 1      AEP      4312.000000  MWh/turbine/year
                     CF          0.328200             %/100
                     Cost   214212.185745    $/turbine/year
                     LCOE        0.049678             $/kWh
              2      AEP      4312.000000  MWh/turbine/year
...                                   ...               ...
              49     LCOE        0.049678             $/kWh
              50     AEP      4312.000000  MWh/turbine/year
                     CF          0.328200             %/100
                     Cost   214212.185745    $/turbine/year
                     LCOE        0.049678             $/kWh

[200 rows x 2 columns]